In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

# Fits to Data With $J=1,3$ Waves
These fits are similar to our final waveset, but add a set of $J=3$ waves to attempt to extract a potential $\rho_3(1690)$ shape. 

Fit Details:
* Spin 1 and 3 waves, with both reflectivities and spin-projections $|m|<3$
  * {$J^P\ell$} = {$1^+S, 1^-P, 1^+D, 3^-F$}
* No isotropic background
* Uses all 4 polarization orientations
* Split data into bins of mass and $-t$
  * 20 MeV wide bins from $1.0 < M_{\omega\pi^0} < 1.8$ 
* Perform 500 randomized fits in each bin of mass and $-t$ independently

## Setup

In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/data-fits/spin-3/no-iso-unconstrained/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

In [ ]:
# load in the preprocessed results 
# t_bins = ["t_0.10-0.16", "t_0.16-0.23", "t_0.23-0.35", "t_0.35-1.00"] #TODO: left out until last 2 bins done
t_bins = ["t_0.10-0.16", "t_0.16-0.23"] 
binned_results = {}
for t_bin in t_bins:
    with open(f"{STUDY_DIR}/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
        data = pkl.load(f)
        binned_results[t_bin] = ResultManager(**data)

binned_results: dict[str, ResultManager]
num_t_bins = len(binned_results)

In [ ]:
for t_bin, results in binned_results.items():
    print(f"Results for t bin: {t_bin}")
    results.summary()

## Analysis

### Standard Plots
These plots are always handy to have

In [ ]:
fig, axs = plt.subplots(1, num_t_bins, figsize=(30, 6))
for i, (t_bin, results) in enumerate(binned_results.items()):    
    results.plot.intensity.jp(ax=axs[i])
    t_low, t_high = results.get_t_edges()    
    axs[i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV²")
    if i != 0:
        axs[i].set_ylabel("")
        axs[i].get_legend().remove()
plt.savefig(f"{STUDY_DIR}/jp_all_t_bins.pdf")    

### D/S Ratio and Phase

In [ ]:
fig, axs = plt.subplots(
    3,
    num_t_bins,    
    sharey="row",
    figsize=(20, 10),
    gridspec_kw={"wspace": 0.0, "hspace": 0.12},
    layout="constrained",
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves=["p1pmS", "m1pmS", "p1ppS", "m1ppS", "m1p0D"])    
    axs[0,i].set_title(rf"${results.get_t_edges()[0]} \leq -t \leq {results.get_t_edges()[1]}$ GeV$^2$")
    if i != 0:
        for ax in axs[:, i]:
            if ax.get_legend() is not None:
                ax.get_legend().remove()
plt.savefig(f"{STUDY_DIR}/ds_ratio_all_t_bins.pdf")

In [ ]:
fig, axs = plt.subplots(
    3,
    num_t_bins,    
    sharey="row",
    figsize=(20, 10),
    gridspec_kw={"wspace": 0.0, "hspace": 0.12},
    layout="constrained",
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves=["p1pmS", "m1pmS", "m1ppS", "p1p0S", "m1p0S"])    
    axs[0,i].set_title(rf"${results.get_t_edges()[0]} \leq -t \leq {results.get_t_edges()[1]}$ GeV$^2$")
    if i != 0:
        for ax in axs[:, i]:
            if ax.get_legend() is not None:
                ax.get_legend().remove()

In [ ]:
fig, axs = plt.subplots(
    3,
    num_t_bins,    
    sharey="row",
    figsize=(20, 10),
    gridspec_kw={"wspace": 0.0, "hspace": 0.12},
    layout="constrained",
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves= ["m1pmS", "p1ppS", "m1ppS", "p1p0S", "m1p0S"])    
    axs[0,i].set_title(rf"${results.get_t_edges()[0]} \leq -t \leq {results.get_t_edges()[1]}$ GeV$^2$")
    if i != 0:
        for ax in axs[:, i]:
            if ax.get_legend() is not None:
                ax.get_legend().remove()

In [ ]:
fig, axs = plt.subplots(
    3,
    num_t_bins,    
    sharey="row",
    figsize=(20, 10),
    gridspec_kw={"wspace": 0.0, "hspace": 0.12},
    layout="constrained",
)
for i, (t_bin, results) in enumerate(binned_results.items()):
    results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves=["p1pmS", "p1ppS", "p1p0S"])    
    axs[0,i].set_title(rf"${results.get_t_edges()[0]} \leq -t \leq {results.get_t_edges()[1]}$ GeV$^2$")
    if i != 0:
        for ax in axs[:, i]:
            if ax.get_legend() is not None:
                ax.get_legend().remove()


### $b_1(1235)$ and $1^{--}$ interference

In [ ]:
fig, axs = plt.subplots(
    2,
    num_t_bins,
    sharex=True,        
    gridspec_kw={"wspace": 0.0, "hspace": 0.07},
    height_ratios=[3, 1],
    figsize=(30, 10),
    layout="constrained",
)
for i, (t_bin, results)in enumerate(binned_results.items()):
    colors = plt.colormaps["Dark2"].colors # type: ignore # match colors to JP plot

    results.plot.phase.mass_phase(
        amp1="p1p0S", amp2="p1mpP",
        amp1_kwargs={"color":colors[2], "alpha": 1.0},
        amp2_kwargs={"color":colors[3], "alpha": 1.0},
        amp_ax=axs[0, i],
        phase_ax=axs[1, i],
    )
    t_low, t_high = results.get_t_edges()
    axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
    if i != 0:
        axs[0,i].set_ylabel("")
        axs[1,i].set_ylabel("")
        axs[0,i].get_legend().remove()
    

max_y=0
for i in range(num_t_bins):
    y_max = axs[0,i].get_ylim()[1]
    max_y = max(max_y, y_max)
for i in range(num_t_bins):
    axs[0,i].set_ylim(0, max_y)


plt.savefig(f"{STUDY_DIR}/mass_phase_all_t_bins.pdf")

### $F_m^{(+)}$ and $1^{-}P_1^{(+)}$ Interference
Note that the P-wave is chosen to be the "reference" wave for these fits as well, as its strongest over the mass spectrum

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

fig, axs = plt.subplots(
    2,
    num_t_bins,
    sharex=True,        
    gridspec_kw={"wspace": 0.0, "hspace": 0.07},
    height_ratios=[3, 1],
    figsize=(30, 10),
    layout="constrained",
)
for i, (t_bin, results)in enumerate(binned_results.items()):
    colors = plt.colormaps["Dark2"].colors # type: ignore # match colors to JP plot

    results.plot.phase.mass_phase(
        amp1="p1p0S", amp2="p3mqF",
        amp1_kwargs={"color":colors[2], "alpha": 1.0},
        amp2_kwargs={"color":colors[7], "alpha": 1.0},
        amp_ax=axs[0, i],
        phase_ax=axs[1, i],
    )
    t_low, t_high = results.get_t_edges()
    axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=20)    
    if i != 0:
        axs[0,i].set_ylabel("")
        axs[1,i].set_ylabel("")
        axs[0,i].get_legend().remove()

    ax_inset = inset_axes(
        axs[0, i],
        width="30%",
        height="40%",
        loc="center right",
        borderpad=2,
    )
    results.plot.phase.mass_phase(
        amp1="p1p0S", amp2="p3mqF",
        amp1_kwargs={"color":colors[2], "alpha": 1.0},
        amp2_kwargs={"color":colors[7], "alpha": 1.0},
        amp_ax=ax_inset,
        phase_ax=axs[1,i],
    )
    ax_inset.set_xlim(1.65, 1.75)

    # zoom in on region 
    indices = results.get_fit_indices(1.64, 1.76)
    values1 = results.fit_df.loc[indices, "p1p0S"]
    values2 = results.fit_df.loc[indices, "p3mqF"]
    ax_inset.set_ylim(
        min(values1.min(), values2.min()) * 0.8,
        max(values1.max(), values2.max()) * 1.2,
    )    

    # TODO: add lines connecting inset to main plot

    ax_inset.legend().remove()
    ax_inset.tick_params(labelsize=12)
    ax_inset.set_title("")  

max_y=0
for i in range(num_t_bins):
    y_max = axs[0,i].get_ylim()[1]
    max_y = max(max_y, y_max)
for i in range(num_t_bins):
    axs[0,i].set_ylim(0, max_y)


# plt.savefig(f"{STUDY_DIR}/mass_phase_all_t_bins.pdf")